In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn import set_config
set_config(display="diagram")
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error



In [ ]:
house = pd.read_csv('housing_prices.csv')

X = house.drop(columns=['Id','SalePrice'])
y = house['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# numeric cols
num_cols = X_train.select_dtypes(exclude=['object']).columns

# ordinal cols
ord_cols = ['ExterQual','ExterCond','KitchenQual','FireplaceQu']
qualities = ["Po", "Fa", "TA", "Gd", "Ex"]
na_qualities = ["missing","Po", "Fa", "TA", "Gd", "Ex"]

# nominal cols
nom_cols = (
    X_train
    .drop(columns=ord_cols)
    .select_dtypes(include=['object'])
    .columns)


# numeric pipeline
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler())
])

# ordinal pipeline
ord_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('encoder', OrdinalEncoder())
])

# nominal pipeline
nom_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('onehot', OneHotEncoder())
])

# preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
        ('num', num_pipe, num_cols),
        ('nom', nom_pipe, nom_cols),
        ('ord', ord_pipe, ord_cols)
])

# model pipeline
model_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('var_threshold', VarianceThreshold()),
    ('mod_feature_sel', SelectFromModel(DecisionTreeRegressor())),
    ('model', LinearRegression())
])


# grid search parameters
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__ord__imputer__strategy': ['most_frequent', 'constant'],
    'preprocessor__ord__imputer__fill_value': ['missing'],
    'preprocessor__ord__encoder__categories': [[qualities for l in range(3)] + [na_qualities]],
    'preprocessor__nom__imputer__strategy': ['most_frequent', 'constant'],
    'preprocessor__nom__onehot__handle_unknown': ['ignore'],
    'var_threshold__threshold': [0, 0.01, 0.02],
    'mod_feature_sel__estimator__max_features': [None, 'auto', 'sqrt', 'log2'],
    'mod_feature_sel__estimator__min_samples_leaf': [1, 2, 4],
    'model__fit_intercept': [True, False]
}

# grid search + cross validation
grid_search = GridSearchCV(model_pipe, param_grid, cv=5, n_jobs=-1, verbose=0)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2nd...
                         'preprocessor__num__imputer__strategy': ['mean',
                                                                  'median'],
                         'preprocessor__ord__encoder__categories': [[['Po',
                                                                      'Fa',
                                                                      'TA',
                                                                      'Gd',
                                                                      'Ex'],
                                                                     ['Po',
                                                                      'Fa',
                                                                      'TA',
                                                                      'Gd',
                                                                      'Ex'],
                                                                     ['Po',
                                                                      'Fa',
                                                                      'TA',
                                                                      'Gd',
                                                                      'Ex'],
                                                                     ['missing',
                                                                      'Po',
                                                                      'Fa',
                                                                      'TA',
                                                                      'Gd',
                                                                      'Ex']]],
                         'preprocessor__ord__imputer__fill_value': ['missing'],
                         'preprocessor__ord__imputer__strategy': ['most_frequent',
                                                                  'constant'],
                         'var_threshold__threshold': [0, 0.01, 0.02]})

In [ ]:
# get predictions
predictions = grid_search.predict(X_train)

# evaluate the model
print(f"R2 score: {r2_score(y_train, predictions)}")
print(f"MSE: {mean_squared_error(y_train, predictions)}")
print(f"MAE: {mean_absolute_error(y_train, predictions)}")


R2 score: 0.8731469091894803
MSE: 781029429.8066794
MAE: 17324.85842838206


In [ ]:
# get predictions
predictions = grid_search.predict(X_test)

# evaluate the model
print(f"R2 score: {r2_score(y_test, predictions)}")
print(f"MSE: {mean_squared_error(y_test, predictions)}")
print(f"MAE: {mean_absolute_error(y_test, predictions)}")


R2 score: 0.658316077106406
MSE: 2359617906.615323
MAE: 21624.315838023194
